In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import zipfile

In [ ]:
extract_file = zipfile.ZipFile('/content/flower_photos.zip', 'r')
extract_file.extractall('/content')
extract_file.close()

In [ ]:
class_names = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

In [ ]:
data_dir = '/content/flower_photos'
filepaths = []
labels = []

folders = os.listdir(data_dir)
for fold in folders:
    foldpath = os.path.join(data_dir, fold)
    filelist = os.listdir(foldpath)
    for file in filelist:
        fpath = os.path.join(foldpath, file)

        filepaths.append(fpath)
        labels.append(fold)

Fseries = pd.Series(filepaths, name='filepaths')
Lseries = pd.Series(labels, name='labels')
df = pd.concat([Fseries, Lseries], axis=1)

In [ ]:
df['labels'].value_counts()

In [ ]:
df['labels'].shape

In [ ]:
X_train_temp, X_test, y_train_temp, y_test = train_test_split(df, df['labels'], test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_temp, y_train_temp, test_size=0.2, random_state=42)

In [ ]:
X_train_resized = np.array([cv2.resize(cv2.imread(img), (IMG_SIZE, IMG_SIZE)) for img in X_train['filepaths']])
X_val_resized = np.array([cv2.resize(cv2.imread(img), (IMG_SIZE, IMG_SIZE)) for img in X_val['filepaths']])
X_test_resized = np.array([cv2.resize(cv2.imread(img), (IMG_SIZE, IMG_SIZE)) for img in X_test['filepaths']])

In [ ]:
X_train = X_train_resized / 255.0
X_val = X_val_resized / 255.0
X_test = X_test_resized / 255.0

In [ ]:
lbc = LabelEncoder()
y_train_encoded = lbc.fit_transform(y_train)
y_val_encoded = lbc.transform(y_val)
y_test_encoded = lbc.transform(y_test)

In [ ]:
y_train_ohc = to_categorical(y_train_encoded)
y_val_ohc = to_categorical(y_val_encoded)
y_test_ohc = to_categorical(y_test_encoded)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())

model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = len(class_names), activation = 'softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(
    X_train,
    y_train_ohc,
    validation_data=(X_val, y_val_ohc),
    epochs=30,
    batch_size=32
)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test_ohc)
print(f'Test accuracy: {test_acc:.2f}')